En primer lugar escribimos el codigo que queremos pero no orientada a objetos.

Empezamos por importar las librerias necesarias.

In [ ]:
import os  # modulo para interactuar con el sistema operativo
import urllib.request  # modulo para realizar solicitudes HTTP
import pandas as pd  # biblioteca para manipulacion de datos en DataFrames
import tarfile  # modulo para manejar archivos tar
from collections import defaultdict  # para crear diccionarios con valores predeterminados
from pathlib import Path  # para manejar rutas de archivos y directorios
import requests  # biblioteca para realizar solicitudes HTTP
from bs4 import BeautifulSoup  # biblioteca para extraer datos de archivos HTML y XML

Creamos una funcion para descargar el archivo.

In [ ]:
def descargar_archivo(url, directorio):
  '''Descarga un archivo desde la URL especificada y lo guarda en el directorio indicado. Si el archivo ya existe, no se descarga de nuevo.'''
  archivo = os.path.join(directorio, os.path.basename(url))  # ruta del archivo
  os.makedirs(os.path.dirname(archivo), exist_ok=True)  # crear el directorio si no existe

  # comprobar si el archivo ya existe antes de descargar
  if not os.path.exists(archivo):
    urllib.request.urlretrieve(url, archivo)  # descargar el archivo
    print(f"Descargado en {archivo}")
  else:
    print(f"El archivo ya existe.")
  return archivo  # retornar la ruta del archivo descargado

Ahora una funcion para descomprimir el archivo descargado.

In [ ]:
def descomprimir_tar_gz(archivo, directorio):
  '''Descomprime un archivo .tar.gz en el directorio especificado.'''
  with tarfile.open(archivo, "r:gz") as tar:
    tar.extractall(path=directorio)  # extraer todos los archivos en el directorio
    print(f"Descomprimido en {directorio}.")

A continuacion una serie de funciones que nos ayudaran a conseguir el objetivo:

1. Leer puntuaciones.

In [4]:
def leer_puntuaciones(directorio):
  '''Lee las puntuaciones de los archivos en el directorio dado y las organiza por identificador.'''
  puntuaciones = defaultdict(list)  # crear un diccionario de listas para almacenar puntuaciones
  for subdir, _, files in os.walk(directorio):
    for file_name in files:
      try:
        # separar el identificador y la puntuación del nombre del archivo
        identificador, puntuacion_str = file_name.split('_')
        puntuacion = int(puntuacion_str.split('.')[0])
        puntuaciones[identificador].append(puntuacion)
      except (ValueError, IndexError):
        continue  # ignorar errores en el procesamiento de nombres de archivos
  return puntuaciones  # retornar el diccionario de puntuaciones

2. Calcular promedio.

In [5]:
def calcular_promedios(puntuaciones):
  '''Calcula los promedios de las puntuaciones para cada identificador y devuelve un conjunto de (identificador, promedio).'''
  return {identificador: sum(scores) / len(scores) for identificador, scores in puntuaciones.items()}  # retornar un diccionario con promedios

3. Obtener los $n$ mejores (sin repeticiones).

In [7]:
def obtener_top_n(promedios, n=10, mejor=True):
  '''Obtiene las top N películas con mejor o peor promedio. Devuelve una lista de (película, promedio) sin repeticiones.'''
  sorted_movies = sorted(promedios.items(), key=lambda x: x[1], reverse=mejor)
  peliculas_unicas = []  # lista para almacenar peliculas unicas
  unicas = set()  # conjunto para rastrear peliculas unicas
  for pelicula, promedio in sorted_movies:
    if pelicula not in unicas:
      peliculas_unicas.append((pelicula, promedio))
      unicas.add(pelicula)  # agregar a la lista de peliculas unicas
    if len(peliculas_unicas) == n:
      break
  return peliculas_unicas  # retornar la lista de peliculas

4. Quitar el texto que no sirve en la url.

In [8]:
def procesar_url(url):
  '''Procesa una URL para eliminar la parte que incluye '/usercomments' y retorna la parte relevante.'''
  return url.split('/usercomments')[0]  # retornar la parte relevante de la url

5. Obtencion de nombres de las peliculas con BeautifulSoup

In [9]:
def obtener_nombre_pelicula(url):
  '''Obtiene el nombre de la película a partir de la url utilizando scraping. Retorna el título o un mensaje de error si no se encuentra.'''
  headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36"
  }  # Definir el User-Agent para la solicitud
  response = requests.get(url, headers=headers)
  if response.status_code == 403:  # comprobar si no hay acceso
    return "Acceso denegado (403 Forbidden)"

  soup = BeautifulSoup(response.content, 'html.parser')
  title_tag = soup.find('title')  # buscar la etiqueta de titulo
  if title_tag:
    return title_tag.text.replace(' - IMDb', '').strip()  # retornar el titulo sin el sufijo de IMDb
  else:
    return "Título no encontrado"

6. Crear dataframe para poner titulo, promedio y url de las peliculas.

In [10]:
def crear_dataframe(peliculas, urls_path):
  '''Crea un DataFrame de pandas a partir de las películas y las urls. Retorna un DataFrame con la url, el nombre de la película y su promedio.'''
  urls = Path(urls_path).read_text().splitlines()
  data = []
  nombres_vistos = set()  # conjunto para rastrear nombres unicos
  for identificador, promedio in peliculas:
    url = procesar_url(urls[int(identificador) - 1])
    nombre_pelicula = obtener_nombre_pelicula(url)  # obtener el nombre de la pelicula a partir de la url
    if nombre_pelicula not in nombres_vistos:  # comprobar si el nombre estaba o no
      nombres_vistos.add(nombre_pelicula)
      data.append({'url': url, 'Película': nombre_pelicula, 'Promedio': promedio})  # agregar al conjunto de datos
  return pd.DataFrame(data)  # retornar el dataframe creado

7. Imprimir peliculas.

In [11]:
def imprimir_peliculas(titulo, peliculas):
  '''Imprime en consola el título y la lista de películas con su promedio y URL.'''
  print(titulo)  # imprimir el titulo
  for pelicula in peliculas:  # recorrer cada pelicula en la lista
    print(f"Película: {pelicula['Película']} || Promedio: {pelicula['Promedio']:.2f} || url: {pelicula['url']}")  # Imprimir detalles de la película

8. Mandar a imprimir las mejores y las peores.

In [12]:
def analizar_peliculas(directorio_pos, directorio_neg, urls_pos, urls_neg):
  '''Coordina el proceso de análisis de películas, incluyendo la lectura de puntuaciones, cálculo de promedios y la impresión de resultados.'''
  calificaciones_pos = leer_puntuaciones(directorio_pos)  # puntuaciones positivas
  calificaciones_neg = leer_puntuaciones(directorio_neg)  # puntuaciones negativas
  promedios_pos = calcular_promedios(calificaciones_pos)  # calcular promedios de puntuaciones positivas
  promedios_neg = calcular_promedios(calificaciones_neg)  # calcular promedios de puntuaciones negativas
  mejores_peliculas = obtener_top_n(promedios_pos, n=10)  # obtener las mejores películas
  peores_peliculas = obtener_top_n(promedios_neg, n=10, mejor=False)  # obtener las peores películas
  lista_mejores = crear_dataframe(mejores_peliculas, urls_pos)  # dataframe de mejores películas
  lista_peores = crear_dataframe(peores_peliculas, urls_neg)  # dataframe de peores películas

  # asegurarse de que la lista de mejores peliculas tenga al menos 10 elementos
  while len(lista_mejores) < 10:
    n_a_row = pd.DataFrame({'url': [''], 'Película': ['N/A'], 'Promedio': [0]})
    lista_mejores = pd.concat([lista_mejores, n_a_row], ignore_index=True)  # agregar fila vacia a la lista

  # asegurarse de que la lista de peores peliculas tenga al menos 10 elementos
  while len(lista_peores) < 10:
    n_a_row = pd.DataFrame({'url': [''], 'Película': ['N/A'], 'Promedio': [0]})
    lista_peores = pd.concat([lista_peores, n_a_row], ignore_index=True)  # agregar fila vacia a la lista

  # imprimir las listas de mejores y peores peliculas
  imprimir_peliculas("Las 10 películas mejor calificadas:\n", lista_mejores.to_dict(orient='records'))
  imprimir_peliculas("Las 10 películas peor calificadas:\n", lista_peores.to_dict(orient='records'))